# Energy Storage Arbitrage in Two Settlement Markets

In [1]:
import os
import torch 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import cvxpy as cp

from main_utils import *

%matplotlib inline

## Data Processing

In [2]:
# Read each timeseries (RTP = Real-Time Price, DAP = Day-Ahead Price, LF = Load Forecast)
for zone in  ['NYC', 'LONGIL', 'WEST', 'NORTH']:
    raw_RTP = pd.read_csv("dataset/nyiso/RTP_"+zone+"_2015_2021.csv", index_col=0)
    raw_DAP = pd.read_csv("dataset/nyiso/DAP_"+zone+"_2015_2021.csv", index_col=0)
    raw_LF  = pd.read_csv("dataset/nyiso/LF_"+zone+"_2015_2021.csv", index_col=0)

    # Prepare the dataset as a dataframe, the target feature renamed as 'OT' to be identified by the model
    # In our case, 'OT' is the 'RTP'
    raw_data = pd.concat([raw_DAP.loc[:,'LBMP ($/MWHr)'], raw_LF.loc[:,'LF'], raw_RTP.loc[:,'LBMP ($/MWHr)']],axis=1).loc['2017-01-01 05:00:00+00:00':]
    raw_data.columns = ['DAP', 'LF', 'OT']
    raw_data.index.names = ['date']
    raw_data.to_csv('dataset/'+zone+'_raw.csv')

    # We perform log tranformation before feeding the dataset into the model to make enhance the performance
    # We only log tranform the price data (RTP and DAP)
    log_data = raw_data.copy(deep=True)
    # log transformation for the forecasting task log10(Y + 1 - min(Y))
    log_data.loc[:,"DAP"] = np.log(raw_data.loc[:,"DAP"] + 1 - min(raw_data.loc[:,"DAP"]))
    log_data.loc[:,"OT"] = np.log(raw_data.loc[:,"OT"] + 1 - min(raw_data.loc[:,"OT"]))
    log_data.to_csv('dataset/'+zone+'_log.csv')


## RTP Forecasting Model
'run_bash_script' will run script file written to train the forecasting model, each zone and model has specific script

In [3]:
for log in [1,0]:
    for model in ['PatchTST', 'DLinear']:
        for zone in ['NYC', 'LONGIL', 'NORTH', 'WEST']:
            if log:
                print('Training model: log_'+model+' for zone: '+zone)
            else:
                print('Training model: '+model+' for zone: '+zone)
            run_script('RTP_Forecasting_Model/', model, zone, log)

Training model: log_PatchTST for zone: NYC
Training model: log_PatchTST for zone: LONGIL
Training model: log_PatchTST for zone: NORTH
Training model: log_PatchTST for zone: WEST
Training model: log_DLinear for zone: NYC
Training model: log_DLinear for zone: LONGIL
Training model: log_DLinear for zone: NORTH
Training model: log_DLinear for zone: WEST
Training model: PatchTST for zone: NYC
Training model: PatchTST for zone: LONGIL
Training model: PatchTST for zone: NORTH
Training model: PatchTST for zone: WEST
Training model: DLinear for zone: NYC
Training model: DLinear for zone: LONGIL
Training model: DLinear for zone: NORTH
Training model: DLinear for zone: WEST


### Result Summary

In [4]:
for log in [1,0]:
    for model in ['PatchTST', 'DLinear']:
        for zone in ['NYC', 'LONGIL', 'NORTH', 'WEST']:
            raw_data = pd.read_csv('dataset/'+zone+'_raw.csv',index_col=0)
            if log:
                pred = np.load('./RTP_Forecasting_Model/results/'+zone+'_log_'+model+'/pred.npy')
                true = np.load('./RTP_Forecasting_Model/results/'+zone+'_log_'+model+'/true.npy')
                pred = np.exp(pred) - 1 + min(raw_data.loc[:,"OT"])
                true = np.exp(true) - 1 + min(raw_data.loc[:,"OT"])
            else:
                pred = np.load('./RTP_Forecasting_Model/results/'+zone+'_raw_'+model+'/pred.npy')
                true = np.load('./RTP_Forecasting_Model/results/'+zone+'_raw_'+model+'/true.npy')          

            MAE, rMAE, rtp_mean = pred_metrics(pred, true)
            if log:
                print('log_'+model+' - '+zone+':')
            else:
                print(model+' - '+zone+':')

            print('     MAE =', MAE)
            print('    rMAE =', rMAE)
            print('RTP mean =', rtp_mean)
            print('        ')


log_PatchTST - NYC:
     MAE = 10.616502
    rMAE = 0.59676063
RTP mean = 42.737732
        
log_PatchTST - LONGIL:
     MAE = 27.083866
    rMAE = 0.7590478
RTP mean = 55.050587
        
log_PatchTST - NORTH:
     MAE = 13.669096
    rMAE = 0.67680895
RTP mean = 23.52548
        
log_PatchTST - WEST:
     MAE = 11.994771
    rMAE = 0.6783312
RTP mean = 31.03292
        
log_DLinear - NYC:
     MAE = 10.970323
    rMAE = 0.61664915
RTP mean = 42.737732
        
log_DLinear - LONGIL:
     MAE = 29.395744
    rMAE = 0.82384014
RTP mean = 55.050587
        
log_DLinear - NORTH:
     MAE = 17.869347
    rMAE = 0.88477933
RTP mean = 23.52548
        
log_DLinear - WEST:
     MAE = 13.590243
    rMAE = 0.76855874
RTP mean = 31.03292
        
PatchTST - NYC:
     MAE = 11.0061
    rMAE = 0.6186603
RTP mean = 42.737717
        
PatchTST - LONGIL:
     MAE = 26.059187
    rMAE = 0.7303301
RTP mean = 55.050697
        
PatchTST - NORTH:
     MAE = 13.414076
    rMAE = 0.66418153
RTP mean = 23.52

## DAM Storage Arbitrage

In [5]:
# Battery Parameters
Power_rating = 0.5
Energy_capacity = 1
Self_discharge_efficiency = 1.00
Charging_Discharging_Efficiency = 0.9
State_of_charge_min = 0
State_of_charge_max = 1
State_of_charge_init = 0
State_of_charge_final = 0
cost_discharge = 0

# Run optimization
DAM_dict = {}
for log in [1,0]:
    DAM_dict[log] = {}
    for model in ['PatchTST', 'DLinear']:
        DAM_dict[log][model] = {}
        for zone in ['NYC', 'LONGIL', 'NORTH', 'WEST']:
            raw_data = pd.read_csv('dataset/'+zone+'_raw.csv',index_col=0)
            if log:
                print('Running model: log_'+model+' for zone: '+zone)
                pred = np.load('./RTP_Forecasting_Model/results/'+zone+'_log_'+model+'/pred.npy')
                true = np.load('./RTP_Forecasting_Model/results/'+zone+'_log_'+model+'/true.npy')
                pred = np.exp(pred) - 1 + min(raw_data.loc[:,"OT"])
                true = np.exp(true) - 1 + min(raw_data.loc[:,"OT"])
            else:
                print('Running model: '+model+' for zone: '+zone)
                pred = np.load('./RTP_Forecasting_Model/results/'+zone+'_raw_'+model+'/pred.npy')
                true = np.load('./RTP_Forecasting_Model/results/'+zone+'_raw_'+model+'/true.npy')

            DAM_dict[log][model][zone] = DAM_Arb(raw_data.iloc[365*4*24+24:,0], true[5::24,:,0], pred[5::24,:,0],
                                                Power_rating, Energy_capacity,Self_discharge_efficiency, Charging_Discharging_Efficiency,
                                                State_of_charge_init, State_of_charge_final, State_of_charge_min, State_of_charge_max, cost_discharge)
            print('\n')



Running model: log_PatchTST for zone: NYC
Restricted license - for non-production use only - expires 2025-11-24
Day 1, Cummulative Profits -75.44655587433239
Day 31, Cummulative Profits -190.19131939527722
Day 61, Cummulative Profits 588.7964326808977
Day 91, Cummulative Profits 806.4661982685227
Day 121, Cummulative Profits 952.4961238637286
Day 151, Cummulative Profits 1253.6341320576228
Day 181, Cummulative Profits 1381.9182878170434
Day 211, Cummulative Profits 1698.983153582688
Day 241, Cummulative Profits 2211.1715393662057
Day 271, Cummulative Profits 2490.6197678323624
Day 301, Cummulative Profits 2778.9772004511333
Day 331, Cummulative Profits 2947.9257194554466
Day 361, Cummulative Profits 3090.3394952316858


Running model: log_PatchTST for zone: LONGIL
Day 1, Cummulative Profits 86.94366766251929
Day 31, Cummulative Profits -58.60889935197247
Day 61, Cummulative Profits 292.2485061173804
Day 91, Cummulative Profits 476.52052406774453
Day 121, Cummulative Profits 533.5341363

## DAM+RTM Profits

### RTM results

In [6]:
# Run optimization
RTM_dict = {}
RTM_profit_dict = {}
for zone in ['NYC', 'LONGIL', 'NORTH', 'WEST']:
    RTM_dict[zone] = {}
    RTM_profit_dict[zone] = {}
    # RTM solution
    RTM_dict[zone] = pd.read_csv('RTM_results/RTM_'+zone+'_2021.csv')
    RTM_profit_dict[zone], RTM_total_profit = Arb_RTM_profit(RTM_dict[zone])
    print('RTM profit for '+zone+' =', RTM_total_profit)

RTM profit for NYC = 10689.836002566823
RTM profit for LONGIL = 31212.208176435128
RTM profit for NORTH = 17117.441691668624
RTM profit for WEST = 13374.422526972578


### Combined Profit

In [7]:
combined_profit_dict = {}
VB_profit_dict = {}
IPM = {}
for log in [1,0]:
    combined_profit_dict[log] = {}
    VB_profit_dict[log] = {}
    for model in ['PatchTST', 'DLinear']:
        combined_profit_dict[log][model] = {}
        VB_profit_dict[log][model] = {}
        for zone in ['NYC', 'LONGIL', 'NORTH', 'WEST']:
            combined_profit_dict[log][model][zone], comb_total_profit,  IPM, VB_profit_dict[log][model][zone] = Arb_comb_profit(DAM_dict[log][model][zone], RTM_dict[zone])
            if log:
                print('Combined Profit for model: log_'+model+' in zone '+zone+' =',comb_total_profit)
                print(f'Improved Profit Margin (IPM) = %{round(IPM, 2)}\n')
            else:
                print('Combined Profit for model: '+model+' in zone '+zone+' =',comb_total_profit)
                print(f'Improved Profit Margin (IPM) = %{round(IPM, 2)}\n')

Combined Profit for model: log_PatchTST in zone NYC = 13875.435245107921
Improved Profit Margin (IPM) = %29.8

Combined Profit for model: log_PatchTST in zone LONGIL = 33848.57007627706
Improved Profit Margin (IPM) = %8.45

Combined Profit for model: log_PatchTST in zone NORTH = 20245.867066068742
Improved Profit Margin (IPM) = %18.28

Combined Profit for model: log_PatchTST in zone WEST = 16836.200496092537
Improved Profit Margin (IPM) = %25.88

Combined Profit for model: log_DLinear in zone NYC = 12580.613592500365
Improved Profit Margin (IPM) = %17.69

Combined Profit for model: log_DLinear in zone LONGIL = 32785.81474679024
Improved Profit Margin (IPM) = %5.04

Combined Profit for model: log_DLinear in zone NORTH = 17318.440665389215
Improved Profit Margin (IPM) = %1.17

Combined Profit for model: log_DLinear in zone WEST = 15289.296828803492
Improved Profit Margin (IPM) = %14.32

Combined Profit for model: PatchTST in zone NYC = 13802.437654799123
Improved Profit Margin (IPM) = %2

### Risk

In [8]:
log = 1
model = 'PatchTST'
for zone in ['NYC', 'LONGIL', 'NORTH', 'WEST']:
    VB_neg, RTM_neg, comb_neg = neg_profit(VB_profit_dict[log][model][zone], RTM_profit_dict[zone], combined_profit_dict[log][model][zone])
    print('zone: '+zone)
    print('No. negative profit days for VB      = '+str(VB_neg))
    print('No. negative profit days for RTM     = '+str(RTM_neg))
    print('No. negative profit days for DAM+RTM = '+str(comb_neg)+'\n')

zone: NYC
No. negative profit days for VB      = 104
No. negative profit days for RTM     = 90
No. negative profit days for DAM+RTM = 59

zone: LONGIL
No. negative profit days for VB      = 153
No. negative profit days for RTM     = 38
No. negative profit days for DAM+RTM = 39

zone: NORTH
No. negative profit days for VB      = 111
No. negative profit days for RTM     = 28
No. negative profit days for DAM+RTM = 27

zone: WEST
No. negative profit days for VB      = 115
No. negative profit days for RTM     = 39
No. negative profit days for DAM+RTM = 33

